---

<div style="text-align: center;">
    <img src="../resources/data-ingestion-cover.png" alt="data-exploration-cover-picture">
</div>

---

# Data Ingestion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/quantumudit/Spend-Estimator/blob/master/notebooks/01_data_exploration.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/quantumudit/Spend-Estimator/master?labpath=notebooks/01_data_exploration.ipynb)

## Introduction

In this notebook, we employ web scraping methods to gather data on the diverse services provided by [Cleanaway](https://www.cleanaway.com.au/contact-us/our-locations/) at various locations throughout Australia.

Once the raw data is collected, we'll engage in data wrangling to refine and reshape it into a more accessible and organized format. This process involves cleaning, transforming, and structuring the data to ensure its accuracy and consistency, ultimately making it suitable for further analysis and visualization.

Upon completing the steps in this notebook, we'll possess a well-prepared dataset that will enable us to uncover insightful patterns and relationships. This refined dataset will serve as the foundation for creating compelling visualizations that effectively convey the story hidden within the data.

## Experiment Setup

In this section, we will establish the required configurations and settings to ensure a seamless data extraction and transformation process. This section involves several crucial steps that lay the foundation for our web scraping and data wrangling tasks. Here's an elaboration of each step:

- **Modules Imports & Setup**: Import and set up various libraries and modules that will aid us in the data ingestion process
- **Execution Path Setup**: Set up the appropriate file path to ensure that our code runs correctly and can access any necessary files or directories
- **Utility Functions & Configurations Setup**: Import and set up essential utility functions and configurations to streamline the data extraction and transformation tasks

### Modules Import and Setup

In [1]:
# Library for operating system related tasks
import os

In [2]:
# Library for RegEx operations
import re

In [3]:
# Libraries for web scraping
import httpx
from selectolax.parser import HTMLParser
from urllib.parse import urljoin

In [4]:
# Other supporting libraries
import random
import time
from datetime import datetime

In [5]:
# Library for data wrangling
import pandas as pd

In [6]:
# Library for pretty print output
from rich.console import Console

console = Console()

### Execution Path Setup

In [7]:
# Current execution path
print(f"Current working directory: {os.path.basename(os.getcwd())}")

# Change to root directory
os.chdir("../")
print(f"Current working directory (Changed): {os.path.basename(os.getcwd())}")

Current working directory: notebooks
Current working directory (Changed): Analyzing-Cleanway-Services


### Utility Functions & Configurations Setup

In [8]:
# Import config.yaml file
from src.constants import CONFIGS

In [12]:
# Import utility function to read YAML files
from src.utils.basic_utils import read_yaml

# Read the data scraper constants from the file
data_scraper = read_yaml(CONFIGS).data_scraper
console.print(data_scraper.to_dict())

[2024-07-14 05:36:35 PM]:ProjectLogger INFO:basic_utils 43 - yaml file: conf\configs.yaml loaded successfully


{
    'root_url': 'https://www.cleanaway.com.au/contact-us/our-locations/',
    'user_agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0',
    'timeout': 100,
    'clear_contents': True,
    'links_data_path': 'data/external/service_links.csv',
    'scraped_data_path': 'data/external/scraped_services.csv'
}

## Data Extraction

In this section, we will perform web scraping to extract data from a website. The process involves several key steps to ensure accurate and efficient data extraction:

- **Fetch Website Response** : Set up various constants and getting the website response by sending a get request to the root url
- **Content Parsing**: Accessing the contents of the root URL and using proper selectors to extract the required information.
- **Handling Pagination**: Using proper CSS selector and logic for navigating to the next page by accessing the next page URL
- **Functionization**: Creating functions to extract required information by handling pagination
- **Service URL Extraction**: Extracting all the service URL and other required information by leveraging the function and pagination logic
- **Sample Service Page Response**: Choosing a sample service URL and getting the website response by sending a get request to the service page URL
- **Sample Service Page Content Parsing**: Extracting required information from the sample service page by using proper selectors
- **Functionization**: Creating a function to extract all the service related information required from the service URL
- **Service Data Extraction**: Extracting all the service details for each service URL available and generating the dataframe

In each of the step we all need to ensure that we are handling any kind of errors and exceptions that might be hindering the required data extracton or, the web scraping process itself.

### Fetch Website Response

Set up various constants and getting the website response by sending a get request to the root url.

In [14]:
# setting up constants
ROOT_URL = (
    data_scraper.root_url
)  # "https://www.cleanaway.com.au/contact-us/our-locations/"
USER_AGENT = data_scraper.user_agent
HEADERS = {"User-Agent": USER_AGENT, "accept-language": "en-US"}
TIMEOUT = data_scraper.timeout

print(f"The root URL is: {ROOT_URL}")
print(f"The request header is: {HEADERS}")
print(f"The timeout is set to: {TIMEOUT} milliseconds")

The root URL is: https://www.cleanaway.com.au/contact-us/our-locations/
The request header is: {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0', 'accept-language': 'en-US'}
The timeout is set to: 100 milliseconds


In [15]:
# Get website response
response = httpx.get(ROOT_URL, headers=HEADERS, timeout=TIMEOUT)
print(f"The status code returned by the website is: {response.status_code}")

[2024-07-14 05:41:20 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www.cleanaway.com.au/contact-us/our-locations/ "HTTP/1.1 200 OK"
The status code returned by the website is: 200


### Content Parsing

In [16]:
# View details of the return object
parsed_html = HTMLParser(response.text)

print(f"The object returned is: {parsed_html}")

print(f"The type of the content is: {type(parsed_html)}")

The object returned is: <HTMLParser chars=323024>
The type of the content is: <class 'selectolax.parser.HTMLParser'>


In [17]:
service_links = parsed_html.css("div.white-box > a")
print(f"Number of services listed on the first page: {len(service_links)}")

Number of services listed on the first page: 9


In [19]:
# Fetch each service URLS
for service in service_links:
    service_url = service.attrs["href"]
    print(f"{service_url}")

https://www.cleanaway.com.au/location/bromelton-energy-and-resource-centre/
https://www.cleanaway.com.au/location/albany/
https://www.cleanaway.com.au/location/lavington/
https://www.cleanaway.com.au/location/alexandra/
https://www.cleanaway.com.au/location/alice-springs/
https://www.cleanaway.com.au/location/albany-2/
https://www.cleanaway.com.au/location/altona/
https://www.cleanaway.com.au/location/armidale/
https://www.cleanaway.com.au/location/cleanaway-artarmon-resource-recovery-centre/


### Handling Pagination

In [ ]:
next_page_url = parsed_html.css_first("li.location-pagination__next a").attrs["href"]
next_page_url_link = urljoin(ROOT_URL, next_page_url)

console.print(f"The next page URL is: [bright_cyan bold]{next_page_url_link}[/]")

### Functionization

In [ ]:
all_service_links = []

In [ ]:
def get_urls(pg_url: str, pg_no: int):
    page_service_links = []
    response = httpx.get(pg_url, headers=HEADERS, timeout=TIMEOUT)

    # Parse the HTML content
    parsed_html = HTMLParser(response.text)

    # Get the services available in the page
    services = parsed_html.css("div.white-box")
    print(f"Total services listed in the page-{pg_no}: {len(services)}")

    # Fetch each service URL
    for service in services:
        service_url = service.css_first("a").attrs["href"]
        service_name = service.css_first("h2").text(strip=True)
        service_address = (
            service.css_first("div.location-info__text")
            .text(strip=True)
            .replace("Address:", "")
        )

        data = {
            "page_number": pg_no,
            "page_url": pg_url,
            "service_url": service_url,
            "service_name": service_name,
            "service_address": service_address,
        }
        page_service_links.append(data)

    # Convert list of dictionaries into dataframe
    df = pd.DataFrame(page_service_links)

    # Return the parsed HTML content and dataframe
    return df, parsed_html

In [ ]:
def get_all_service_links(page_url: str, page_num: int = 1):
    # Generate random sleep time
    sleep_sec = random.randint(1, 5)

    # Write the product info and get the page HTML in variable
    df, content = get_urls(page_url, page_num)
    all_service_links.append(df)
    print(f"Page-{page_num} services links extracted")
    time.sleep(sleep_sec)

    # Increase the page number
    page_num += 1

    # recursively call to extract all product links
    next_page_element = content.css_first("li.location-pagination__next a")

    if next_page_element is not None:
        next_page_url = urljoin(ROOT_URL, next_page_element.attrs["href"])
        get_all_service_links(next_page_url, page_num)

### Service URL Extraction

In [ ]:
get_all_service_links(page_url=ROOT_URL)

In [ ]:
len(all_service_links)

In [ ]:
services_links_df = pd.concat(all_service_links, axis=0, ignore_index=True)
services_links_df

### Parse Single Service Details

In this section, we'll pick a single service link and scrape the required details from the website.

In [36]:
service = service_links[1]

service_url = service.attrs["href"]
console.print(
    f"The service URL to scrape details is: [bright_cyan bold]{service_url}[/]"
)

The service URL to scrape details is: https://www.cleanaway.com.au/location/albany/

In [22]:
# Fetch response from service url
service_response = httpx.get(service_url, headers=HEADERS, timeout=TIMEOUT)
print(f"The status code returned by the website is: {service_response.status_code}")

[2024-07-14 05:44:10 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www.cleanaway.com.au/location/albany/ "HTTP/1.1 200 OK"
The status code returned by the website is: 200


In [23]:
# View details of the return object
svc_html = HTMLParser(service_response.text)
print(f"The object returned is: {svc_html}")

The object returned is: <HTMLParser chars=308182>


In [25]:
# Get service name
service_name = svc_html.css_first("div.location-box h1").text(strip=True)
console.print(f"The service name is: [bright_green bold]{service_name}[/]")

The service name is: Albany Material Recovery Facility

In [35]:
# Get service address
address = svc_html.css_first("div.location-box div.info-block:first-of-type p a").text(
    strip=True
)
console.print(f"The service address is: [bright_green bold]{address}[/]")

The service address is: 2-16 Cuming Road & 37 Maxwell Street, Albany, WA, 6330

In [37]:
# Get service location coordinates
address_url = svc_html.css_first(
    "div.location-box div.info-block:first-of-type p a"
).attrs["href"]

console.print(f"The service address URL is: [bright_cyan bold]{address_url}[/]")

The service address URL is: http://maps.google.com/maps/place/?q=-35.01078332713777,117.86331638131479

In [39]:
lat_long_pattern = re.compile(r".*\?q=(.*)\,(.*)")
lat_long_matches = lat_long_pattern.match(address_url)

latitude = lat_long_matches.group(1)
longitude = lat_long_matches.group(2)

console.print(
    f"The service coordinates is: [bright_magenta bold]({latitude}, {longitude})[/]"
)

The service coordinates is: (-35.01078332713777, 117.86331638131479)

In [41]:
# Fetch services offered
if (
    "services"
    in svc_html.css_first(
        "div.location-box div.info-block:last-of-type div.info-block__title"
    )
    .text(strip=True)
    .lower()
):
    services = svc_html.css_first(
        "div.location-box div.info-block:last-of-type div.info-block__desc p"
    ).text(strip=True)
else:
    services = "Miscellaneous"

console.print(f"The services available are: [bright_green bold]{services}[/]")

The services available are: Solid waste services, Office

### Scrape Service Details from Service Links

Using the scraped service links to extract detailed information.

In [49]:
def get_service_details(svc_url: str, svc_nm_crd: str, svc_adrs_crd: str):
    response = httpx.get(svc_url, headers=HEADERS, timeout=TIMEOUT)

    # Parse the HTML content
    html = HTMLParser(response.text)

    # CSS selectors
    service_name_css = "div.location-box h1"
    address_css = "div.location-box div.info-block:first-of-type p a"
    last_info_block = "div.location-box div.info-block:last-of-type"
    services_title_css = f"{last_info_block} div.info-block__title"
    services_offered_css = f"{last_info_block} div.info-block__desc p"

    # Function to fetch either text or, URL address
    def fetch(selector, output="text"):
        if html.css_first(selector):
            if output == "text":
                output = html.css_first(selector).text(strip=True)
            elif output == "link":
                output = html.css_first(selector).attrs["href"]
        else:
            output = None
        return output

    # Service and service address
    service_name = (
        svc_nm_crd if fetch(service_name_css) is None else fetch(service_name_css)
    )
    service_address = svc_adrs_crd if fetch(address_css) is None else fetch(address_css)

    # Address URL
    address_url = fetch(address_css, "link")

    # RegEx pattern match for latitude and longitudes
    if address_url:
        lat_long_pattern = re.compile(r".*\?q=(.*?)\,+(.*)")
        matches = lat_long_pattern.match(address_url)
        lat, long = matches.group(1), matches.group(2)
    else:
        lat, long = None, None

    # Services offered
    if fetch(services_offered_css):
        services_offered = (
            fetch(services_offered_css)
            if "Services" in fetch(services_title_css)
            else "Miscellaneous"
        )
    else:
        services_offered = "Miscellaneous"

    # Get the product details in data class
    service_details = dict(
        service_name=service_name,
        address=service_address,
        latitude=lat,
        longitude=long,
        services_offered=services_offered,
        details_url=svc_url,
        scrape_ts=datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    )

    # Write scraped info into CSV file
    return service_details

In [51]:
all_service_details = []

print("Services scraping started")
print("No. of Service Details Scraped: ", end=" ")
for idx, row in services_links_df.iterrows():
    service_url = row["service_url"]
    service_name_card = row["service_name"]
    service_address_card = row["service_address"]

    # Generate random sleep time
    sleep_sec = random.randint(1, 3)

    # write services data to csv file
    svc_details = get_service_details(
        service_url, service_name_card, service_address_card
    )
    all_service_details.append(svc_details)
    print(idx + 1, end=", ")
    time.sleep(sleep_sec)

print("\nScraping Completed")

Services scraping started
No. of Service Details Scraped:  [2024-07-14 08:42:17 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www.cleanaway.com.au/location/bromelton-energy-and-resource-centre/ "HTTP/1.1 200 OK"
1, [2024-07-14 08:42:19 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www.cleanaway.com.au/location/albany/ "HTTP/1.1 200 OK"
2, [2024-07-14 08:42:22 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www.cleanaway.com.au/location/lavington/ "HTTP/1.1 200 OK"
3, [2024-07-14 08:42:25 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www.cleanaway.com.au/location/alexandra/ "HTTP/1.1 200 OK"
4, [2024-07-14 08:42:33 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www.cleanaway.com.au/location/alice-springs/ "HTTP/1.1 200 OK"
5, [2024-07-14 08:42:40 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www.cleanaway.com.au/location/albany-2/ "HTTP/1.1 200 OK"
6, [2024-07-14 08:42:46 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://www

In [52]:
services_df = pd.DataFrame(all_service_details)
services_df

,service_name,address,latitude,longitude,services_offered,details_url,scrape_ts
0,Bromelton Energy & Resource Centre,"Lot 1 Beaudesert-Boonah Road, Bromelton QLD",-27.975059190727386,152.92509487274305,Miscellaneous,https://www.cleanaway.com.au/location/bromelto...,2024-07-14 20:42:17
1,Albany Material Recovery Facility,"2-16 Cuming Road & 37 Maxwell Street, Albany, ...",-35.01078332713777,117.86331638131479,"Solid waste services, Office",https://www.cleanaway.com.au/location/albany/,2024-07-14 20:42:19
2,Albury-Wodonga Solid Waste Services,"26 Reiff St, Lavington, NSW, 2641",-36.037913,146.964581,Solid waste services,https://www.cleanaway.com.au/location/lavington/,2024-07-14 20:42:22
3,Alexandra Solid Waste Services,"4-8 Station Street, Alexandra, VIC, 3714",-37.183192,145.712031,Solid waste services,https://www.cleanaway.com.au/location/alexandra/,2024-07-14 20:42:25
4,Cleanaway Alice Springs Solid Waste Depot,"6 Wilkinson Street, Ciccone, NT, 0870",-23.700021,133.871345,Solid waste services,https://www.cleanaway.com.au/location/alice-sp...,2024-07-14 20:42:33
...,...,...,...,...,...,...,...
209,Cleanaway Wodonga Industrial Waste Services,"2/9 Romet Road, Wodonga, VIC, 3690",-36.11166500739213,146.85815657030102,Industrial waste services,https://www.cleanaway.com.au/location/wodonga/,2024-07-14 21:10:53
210,Cleanaway Wyong Solid Waste Depot,"1 Amsterdam Circuit, Wyong, NSW, 2259",-33.264526357566275,151.45145710619676,"Solid waste services, Office",https://www.cleanaway.com.au/location/wyong/,2024-07-14 21:11:01
211,Cleanaway Yatala Liquid Waste Services,"29 Binary St, Yatala, QLD, 4207",-27.757649419169535,153.23397267269092,Liquid waste services,https://www.cleanaway.com.au/location/yatala/,2024-07-14 21:11:08
212,Cleanaway Young Solid Waste Services,"20 Chillingworks Rd, Young, NSW, 2594",-34.30134824214955,148.2713198766994,Solid waste services,https://www.cleanaway.com.au/location/young-nsw/,2024-07-14 21:11:15


In [53]:
### Exporting the dataframe
services_df.to_csv("services_scraped_temp.csv", index=False)

## Data Profiling

- Dataframe details
- Field details
- Duplicated entry check
- Unique values check

### Data Structure Analysis

This section is devoted to an initial exploration of the structure of our dataframe. 
Our focus will be on various aspects of the dataframe, such as the number of columns, 
presence of null values, shape, and attributes of the individual fields within the dataframe. 
The goal of this analysis is to identify any potential challenges or issues that may arise 
during further analysis.

We can break down our exploration into two parts:

- **Dataframe Overview**: To gain an understanding of overall dataframe attributes
- **Field Attributes**: To examine the attributes of individual fields within the 
  dataframe

With this framework in mind, let's proceed with a closer look at the structure of our dataframe.

In [77]:
from src.utils.analysis_utils import dataframe_structure, datatype_details
from src.utils.basic_utils import dict_to_table

#### Dataframe Details

This section is a crucial one and it provides a high-level overview of the dataframe being analyzed. This section includes important information about the size and structure of the data, as well as any missing or null values in the data. The details of the dataframe that we'll get are as follows:

- **Dimensions**: The number of rows and columns in the dataframe.
- **Shape**: The shape of the dataframe, represented as a tuple (rows, columns).
- **Row Count**: The number of rows in the dataframe.
- **Column Count**: The number of columns in the dataframe.
- **Total Datapoints**: The total number of data points in the dataframe, calculated as the number of rows multiplied by the number of columns.
- **Null Datapoints**: The number of missing or null values in the dataframe.
- **Non-Null Datapoints**: The number of non-missing or non-null values in the dataframe.
- **Total Memory Usage**: The total memory usage of the dataframe, represented in bytes.
- **Average Memory Usage**: The average memory usage of each data point in the dataframe, represented in bytes.

This section provides a quick reference for the dataframe, and helps to identify any potential issues with the data that may need to be addressed in the cleaning process. By having a clear and concise overview of the dataframe, it's easier to move forward with the data cleaning and analysis process.

In [78]:
# Get the dataframe structure details
df_structure = dataframe_structure(services_df)
print(dict_to_table(df_structure, "Dataframe Structure"))

        Dataframe Structure        
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Dataframe Attributes ┃    Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Dimensions           │        2 │
│ Shape                │ (232, 7) │
│ Row Count            │      232 │
│ Column Count         │        7 │
│ Total Datapoints     │     1624 │
│ Null Datapoints      │        2 │
│ Non-Null Datapoints  │     1622 │
│ Total Memory Usage   │   141797 │
│ Average Memory Usage │  17725.0 │
└──────────────────────┴──────────┘

#### Field Details

This section provides a detailed view of the columns in the dataframe, and includes important information that can help with the data cleaning and analysis process. This section is typically used to get an understanding of the data types and distributions of the columns, and to identify any missing or null values that may need to be addressed.

We can use the `info()` method in `pandas` to quickly get an understanding of the structure and distribution of the data, and identify any issues that may need to be addressed in the cleaning process.

Additionally, we'll use the function `datatype_details()` to get a datatype details of the dataframe.

Some of the key information that we'll get from this section are as follows:

- **Column Names**: A list of the names of all the columns in the dataframe, which can help you to identify any columns that may need to be renamed for clarity or consistency.
- **Data types**: The datatype of each column in the dataframe, such as integer, float, string, etc. The datatype information is important because it determines how the data can be analyzed and manipulated.
- **Non-Null Count**: The number of non-null or non-missing values for each column in the dataframe. This information can help you to determine if there are any columns that have a high percentage of missing values, which may need to be handled differently in the cleaning process.
- **Null Count**: The number of null or missing values for each column in the dataframe. This information can help you to determine if there are any columns that have a high number of missing values, which may need to be handled differently in the cleaning process.
- **Memory Usage**: The memory usage of each column in the dataframe, represented in bytes. This information can be useful in determining if there are any columns that are using a large amount of memory, which may need to be optimized for performance.

In [81]:
# Get field details
services_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   service_name      232 non-null    object
 1   address           232 non-null    object
 2   latitude          231 non-null    object
 3   longitude         231 non-null    object
 4   services_offered  232 non-null    object
 5   details_url       232 non-null    object
 6   scrape_ts         232 non-null    object
dtypes: object(7)
memory usage: 138.5 KB


In [82]:
# Get details of the datatypes
datatype_details(services_df)

'There are 7 fields with object datatype'

### Data Quality Analysis

The purpose of this section is to evaluate the overall quality of the data by checking for any missing or null values, duplicate entries, and any other potential issues that may impact the accuracy or reliability of our analysis. 

This step is crucial in ensuring that the data we are working with is valid and can be used to draw meaningful conclusions. We will also be identifying and addressing any issues that are found, in order to improve the overall quality of the data.

From our data structure analysis step, we made our data relatively clean and now, we need to perform the following two checks on the dataset:

- **Removal of Unnecessary Columns**: To eliminate irrelevant columns from the dataset.
- **Duplicated Entry Checks**: To identify and remove duplicate rows in the dataset.
- **Unique Value Checks**: To verify the uniqueness of values in specific columns of the dataset.

In [83]:
from src.utils.analysis_utils import object_fields_count_stats

#### Duplicated Entry Checks

Now, we will be analyzing the dataframe for any duplicate entries. This will involve identifying and removing any duplicate rows in the dataframe to ensure that our data is clean and accurate. 

This step is important as duplicate data can lead to inaccurate or misleading results in our analysis. We will be using pandas inbuilt methods to check for duplicate rows and remove them as necessary.

In [106]:
# Create boolean mask to get duplicated entries
dup_mask = services_df.drop(columns="scrape_ts").duplicated()

# Apply the mask over dataframe
dup_entries = services_df[dup_mask]

# Print the results
print(f"There are {len(dup_entries)} duplicated entries in the dataframe")

There are 18 duplicated entries in the dataframe

#### Unique Value Checks

In this section, we will be checking the number of unique values present in each object type column in the data set. Object type columns are those columns in the data set that contain string values, as opposed to numerical or boolean values. 

This check is important as it helps us understand the diversity of the data in each column and helps us identify columns that may contain redundant or irrelevant information.

To perform this check, we will use the `object_fields_count_stats()` utility function that will provide not only unique values but, some other valuable count related statistics.

The different info that the function provides are as follows:

- `total_rows`: The total number of rows in a specific column.
- `null_rows`: The number of rows in a specific column containing null or missing values.
- `not_null_rows`: The number of rows in a specific column with non-null or present values.
- `unique_item_count`: The count of unique items in a specific column.
- `distinct_item_count`: The count of items that appear only once in a specific column.

In [85]:
# Get count statistics for object type fields
count_stats_df = object_fields_count_stats(services_df)

# View count statistics
count_stats_df

,total_rows,null_rows,not_null_rows,unique_item_count,distinct_item_count
column,,,,,
service_name,232,0,232,214,205
address,232,0,232,209,195
latitude,232,1,231,213,204
longitude,232,1,231,213,204
services_offered,232,0,232,19,9
details_url,232,0,232,214,205
scrape_ts,232,0,232,232,232


## Data Transformation

In [111]:
services_df_unq = services_df.drop(columns="scrape_ts").drop_duplicates()
services_df = services_df.loc[services_df_unq.index]

In [112]:
# Create boolean mask to get duplicated entries
dup_mask = services_df.drop(columns="scrape_ts").duplicated()

# Apply the mask over dataframe
dup_entries = services_df[dup_mask]

# Print the results
print(f"There are {len(dup_entries)} duplicated entries in the dataframe")

There are 0 duplicated entries in the dataframe

In [87]:
services_df[services_df["longitude"].isna()]

,service_name,address,latitude,longitude,services_offered,details_url,scrape_ts
154,Cleanaway Narangba Vacuum Truck Liquid Waste S...,"109 Potassium Street, Narangba, QLD, 4504",None,None,Miscellaneous,https://www.cleanaway.com.au/location/narangba/,2024-07-11 14:32:06


In [91]:
from src.utils.basic_utils import get_lat_long

ImportError: cannot import name 'get_lat_long' from 'src.utils.basic_utils' (d:\Projects\Github\04_Data_Analytics\Analyzing-Cleanway-Services\src\utils\basic_utils.py)

In [92]:
def get_lat_long(address):
    url = f"https://nominatim.openstreetmap.org/?q={address}&format=json"
    response = httpx.get(url)
    if response.status_code == 200:
        data = response.json()
        return {"lat": float(data[0]["lat"]), "long": float(data[0]["lon"])}
    else:
        return "Error: Unable to retrieve location information"

In [93]:
def apply_function(row):
    if pd.isnull(row["latitude"]) or pd.isnull(row["longitude"]):
        coordinates = get_lat_long(row["address"])
        latitude, longitude = coordinates["lat"], coordinates["long"]
        return latitude, longitude
    else:
        return row["latitude"], row["longitude"]

In [94]:
services_df["latitude"], services_df["longitude"] = zip(
    *services_df.apply(apply_function, axis=1)
)

[2024-07-11 06:46:45 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://nominatim.openstreetmap.org/?q=109%20Potassium%20Street,%20Narangba,%20QLD,%204504&format=json "HTTP/1.1 200 OK"


In [95]:
# Get the dataframe structure details
df_structure = dataframe_structure(services_df)
print(dict_to_table(df_structure, "Dataframe Structure"))

        Dataframe Structure        
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Dataframe Attributes ┃    Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Dimensions           │        2 │
│ Shape                │ (232, 7) │
│ Row Count            │      232 │
│ Column Count         │        7 │
│ Total Datapoints     │     1624 │
│ Null Datapoints      │        0 │
│ Non-Null Datapoints  │     1624 │
│ Total Memory Usage   │   142540 │
│ Average Memory Usage │  17818.0 │
└──────────────────────┴──────────┘

In [96]:
services_df["address"]

0            Lot 1 Beaudesert-Boonah Road, Bromelton QLD
1      2-16 Cuming Road & 37 Maxwell Street, Albany, ...
2                      26 Reiff St, Lavington, NSW, 2641
3               4-8 Station Street, Alexandra, VIC, 3714
4                  6 Wilkinson Street, Ciccone, NT, 0870
                             ...                        
227                   2/9 Romet Road, Wodonga, VIC, 3690
228                1 Amsterdam Circuit, Wyong, NSW, 2259
229                      29 Binary St, Yatala, QLD, 4207
230                20 Chillingworks Rd, Young, NSW, 2594
231                510 Summerhill Road, Wollert VIC 3750
Name: address, Length: 232, dtype: object

In [98]:
services_df["state"] = services_df["address"].str.extract(
    r".+?((?:[A-Z]{2,3}|Victoria|Vic|Western Australia))", expand=False
)
services_df["postcode"] = services_df["address"].str.extract(
    r".* (\d{4})", expand=False
)

In [99]:
services_df["state"].unique()

array(['QLD', 'WA', 'NSW', 'VIC', 'NT', 'Vic', 'Victoria', 'ACT', 'SA',
       'TAS', 'Western Australia'], dtype=object)

In [100]:
services_df["state"] = (
    services_df["state"]
    .str.replace(r"Vic(?:toria)?", "VIC", regex=True)
    .str.replace("Western Australia", "WA")
)

In [102]:
# Get the dataframe structure details
df_structure = dataframe_structure(services_df)
print(dict_to_table(df_structure, "Dataframe Structure"))

        Dataframe Structure        
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Dataframe Attributes ┃    Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Dimensions           │        2 │
│ Shape                │ (232, 9) │
│ Row Count            │      232 │
│ Column Count         │        9 │
│ Total Datapoints     │     2088 │
│ Null Datapoints      │        3 │
│ Non-Null Datapoints  │     2085 │
│ Total Memory Usage   │   170467 │
│ Average Memory Usage │  17047.0 │
└──────────────────────┴──────────┘

In [113]:
# Get count statistics for object type fields
count_stats_df = object_fields_count_stats(services_df)

# View count statistics
count_stats_df

,total_rows,null_rows,not_null_rows,unique_item_count,distinct_item_count
column,,,,,
service_name,214,0,214,214,214
address,214,0,214,209,204
latitude,214,0,214,214,214
longitude,214,0,214,214,214
services_offered,214,0,214,19,9
details_url,214,0,214,214,214
scrape_ts,214,0,214,214,214
state,214,0,214,8,0
postcode,214,1,213,158,121


In [114]:
services_df[services_df["postcode"].isna()]

,service_name,address,latitude,longitude,services_offered,details_url,scrape_ts,state,postcode
0,Bromelton Energy & Resource Centre,"Lot 1 Beaudesert-Boonah Road, Bromelton QLD",-27.975059190727386,152.92509487274305,Miscellaneous,https://www.cleanaway.com.au/location/bromelto...,2024-07-11 14:21:52,QLD,NaN


In [ ]:
from src.utils.basic_utils import get_postcode

In [101]:
def get_postcode(address):
    postcode_pattern = r".* (\d{4})"
    url = f"https://nominatim.openstreetmap.org/?q={address}&format=json"
    response = httpx.get(url)
    if response.status_code == 200:
        data = response.json()
        for loc in data:
            loc_name = loc["display_name"]
            if re.findall(postcode_pattern, loc_name):
                return re.findall(postcode_pattern, loc_name)
    else:
        return "Error: Unable to retrieve location information"

In [117]:
def apply_postcode_function(row):
    if pd.isnull(row["postcode"]):
        postcode = get_postcode(row["address"])[0]
        return postcode
    else:
        return row["postcode"]

In [118]:
services_df["postcode"] = services_df.apply(apply_postcode_function, axis=1)

[2024-07-11 07:14:19 PM]:httpx INFO:_client 1026 - HTTP Request: GET https://nominatim.openstreetmap.org/?q=Lot%201%20Beaudesert-Boonah%20Road,%20Bromelton%20QLD&format=json "HTTP/1.1 200 OK"


In [119]:
services_df[services_df["postcode"].isna()]

,service_name,address,latitude,longitude,services_offered,details_url,scrape_ts,state,postcode


In [120]:
# Get the dataframe structure details
df_structure = dataframe_structure(services_df)
print(dict_to_table(df_structure, "Dataframe Structure"))

        Dataframe Structure        
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Dataframe Attributes ┃    Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Dimensions           │        2 │
│ Shape                │ (214, 9) │
│ Row Count            │      214 │
│ Column Count         │        9 │
│ Total Datapoints     │     1926 │
│ Null Datapoints      │        0 │
│ Non-Null Datapoints  │     1926 │
│ Total Memory Usage   │   159204 │
│ Average Memory Usage │  15920.0 │
└──────────────────────┴──────────┘

In [122]:
services_df["services_offered"].unique()

array(['Miscellaneous', 'Solid waste services, Office',
       'Solid waste services',
       'Liquid waste services, Industrial waste services',
       'Liquid waste services', 'Industrial waste services', 'Office',
       'Office, Public Waste Drop-off',
       'Solid waste services, Transfer Station', 'Cleanaway Daniels',
       'Transfer Station, Office', 'Transfer Station',
       'Industrial waste services, Office',
       'Solid waste services, Liquid waste services',
       'Solid waste services, Industrial waste services',
       'Liquid waste services, Office',
       'Hazardous Soil Treatment, Industrial Waste Services Depot',
       'Transfer Station, Office, Public Waste Drop-off, Commercial Waste Drop-off',
       'Liquid waste services, Industrial waste services, Office'],
      dtype=object)

In [123]:
custom_index_col = pd.RangeIndex(
    start=1000, stop=1000 + len(services_df), step=1, name="id"
)

services_df.index = custom_index_col

services_df.index = "SVC" + services_df.index.astype("string")

services_df = services_df.reset_index()

In [124]:
services_df.head()

,id,service_name,address,latitude,longitude,services_offered,details_url,scrape_ts,state,postcode
0,P1000,Bromelton Energy & Resource Centre,"Lot 1 Beaudesert-Boonah Road, Bromelton QLD",-27.975059190727386,152.92509487274305,Miscellaneous,https://www.cleanaway.com.au/location/bromelto...,2024-07-11 14:21:52,QLD,4285
1,P1001,Albany Material Recovery Facility,"2-16 Cuming Road & 37 Maxwell Street, Albany, ...",-35.01078332713777,117.86331638131479,"Solid waste services, Office",https://www.cleanaway.com.au/location/albany/,2024-07-11 14:22:03,WA,6330
2,P1002,Albury-Wodonga Solid Waste Services,"26 Reiff St, Lavington, NSW, 2641",-36.037913,146.964581,Solid waste services,https://www.cleanaway.com.au/location/lavington/,2024-07-11 14:22:07,NSW,2641
3,P1003,Alexandra Solid Waste Services,"4-8 Station Street, Alexandra, VIC, 3714",-37.183192,145.712031,Solid waste services,https://www.cleanaway.com.au/location/alexandra/,2024-07-11 14:22:15,VIC,3714
4,P1004,Cleanaway Alice Springs Solid Waste Depot,"6 Wilkinson Street, Ciccone, NT, 0870",-23.700021,133.871345,Solid waste services,https://www.cleanaway.com.au/location/alice-sp...,2024-07-11 14:22:18,NT,0870


In [125]:
services_df["services_offered"] = services_df["services_offered"].str.split(",")
services_df

,id,service_name,address,latitude,longitude,services_offered,details_url,scrape_ts,state,postcode
0,P1000,Bromelton Energy & Resource Centre,"Lot 1 Beaudesert-Boonah Road, Bromelton QLD",-27.975059190727386,152.92509487274305,[Miscellaneous],https://www.cleanaway.com.au/location/bromelto...,2024-07-11 14:21:52,QLD,4285
1,P1001,Albany Material Recovery Facility,"2-16 Cuming Road & 37 Maxwell Street, Albany, ...",-35.01078332713777,117.86331638131479,"[Solid waste services, Office]",https://www.cleanaway.com.au/location/albany/,2024-07-11 14:22:03,WA,6330
2,P1002,Albury-Wodonga Solid Waste Services,"26 Reiff St, Lavington, NSW, 2641",-36.037913,146.964581,[Solid waste services],https://www.cleanaway.com.au/location/lavington/,2024-07-11 14:22:07,NSW,2641
3,P1003,Alexandra Solid Waste Services,"4-8 Station Street, Alexandra, VIC, 3714",-37.183192,145.712031,[Solid waste services],https://www.cleanaway.com.au/location/alexandra/,2024-07-11 14:22:15,VIC,3714
4,P1004,Cleanaway Alice Springs Solid Waste Depot,"6 Wilkinson Street, Ciccone, NT, 0870",-23.700021,133.871345,[Solid waste services],https://www.cleanaway.com.au/location/alice-sp...,2024-07-11 14:22:18,NT,0870
...,...,...,...,...,...,...,...,...,...,...
209,P1209,Cleanaway Wodonga Industrial Waste Services,"2/9 Romet Road, Wodonga, VIC, 3690",-36.11166500739213,146.85815657030102,[Industrial waste services],https://www.cleanaway.com.au/location/wodonga/,2024-07-11 14:37:46,VIC,3690
210,P1210,Cleanaway Wyong Solid Waste Depot,"1 Amsterdam Circuit, Wyong, NSW, 2259",-33.264526357566275,151.45145710619676,"[Solid waste services, Office]",https://www.cleanaway.com.au/location/wyong/,2024-07-11 14:37:49,NSW,2259
211,P1211,Cleanaway Yatala Liquid Waste Services,"29 Binary St, Yatala, QLD, 4207",-27.757649419169535,153.23397267269092,[Liquid waste services],https://www.cleanaway.com.au/location/yatala/,2024-07-11 14:37:53,QLD,4207
212,P1212,Cleanaway Young Solid Waste Services,"20 Chillingworks Rd, Young, NSW, 2594",-34.30134824214955,148.2713198766994,[Solid waste services],https://www.cleanaway.com.au/location/young-nsw/,2024-07-11 14:37:57,NSW,2594


In [127]:
df_exploded = services_df.explode("services_offered")
df_exploded.head()

,id,service_name,address,latitude,longitude,services_offered,details_url,scrape_ts,state,postcode
0,P1000,Bromelton Energy & Resource Centre,"Lot 1 Beaudesert-Boonah Road, Bromelton QLD",-27.975059190727386,152.92509487274305,Miscellaneous,https://www.cleanaway.com.au/location/bromelto...,2024-07-11 14:21:52,QLD,4285
1,P1001,Albany Material Recovery Facility,"2-16 Cuming Road & 37 Maxwell Street, Albany, ...",-35.01078332713777,117.86331638131479,Solid waste services,https://www.cleanaway.com.au/location/albany/,2024-07-11 14:22:03,WA,6330
1,P1001,Albany Material Recovery Facility,"2-16 Cuming Road & 37 Maxwell Street, Albany, ...",-35.01078332713777,117.86331638131479,Office,https://www.cleanaway.com.au/location/albany/,2024-07-11 14:22:03,WA,6330
2,P1002,Albury-Wodonga Solid Waste Services,"26 Reiff St, Lavington, NSW, 2641",-36.037913,146.964581,Solid waste services,https://www.cleanaway.com.au/location/lavington/,2024-07-11 14:22:07,NSW,2641
3,P1003,Alexandra Solid Waste Services,"4-8 Station Street, Alexandra, VIC, 3714",-37.183192,145.712031,Solid waste services,https://www.cleanaway.com.au/location/alexandra/,2024-07-11 14:22:15,VIC,3714


In [129]:
df_exploded["services_offered"].unique()

array(['Miscellaneous', 'Solid waste services', ' Office',
       'Liquid waste services', ' Industrial waste services',
       'Industrial waste services', 'Office', ' Public Waste Drop-off',
       ' Transfer Station', 'Cleanaway Daniels', 'Transfer Station',
       ' Liquid waste services', 'Hazardous Soil Treatment',
       ' Industrial Waste Services Depot', ' Commercial Waste Drop-off'],
      dtype=object)

In [130]:
df_exploded["services_offered"] = df_exploded["services_offered"].str.strip()

In [131]:
df_exploded["services_offered"].unique()

array(['Miscellaneous', 'Solid waste services', 'Office',
       'Liquid waste services', 'Industrial waste services',
       'Public Waste Drop-off', 'Transfer Station', 'Cleanaway Daniels',
       'Hazardous Soil Treatment', 'Industrial Waste Services Depot',
       'Commercial Waste Drop-off'], dtype=object)

In [134]:
df_exploded["latitude"] = pd.to_numeric(df_exploded["latitude"], errors="coerce")
df_exploded["longitude"] = pd.to_numeric(df_exploded["longitude"], errors="coerce")
df_exploded["scrape_ts"] = pd.to_datetime(df_exploded["scrape_ts"])

In [135]:
df_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 249 entries, 0 to 213
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                249 non-null    string        
 1   service_name      249 non-null    object        
 2   address           249 non-null    object        
 3   latitude          249 non-null    float64       
 4   longitude         249 non-null    float64       
 5   services_offered  249 non-null    object        
 6   details_url       249 non-null    object        
 7   scrape_ts         249 non-null    datetime64[ns]
 8   state             249 non-null    object        
 9   postcode          249 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(6), string(1)
memory usage: 21.4+ KB


In [136]:
# check duplicates
# rearrange columns
# add description to the notebook